# DeepLabCut for your standard (single animal) projects!

Some useful links:

- [DeepLabCut's GitHub: github.com/DeepLabCut/DeepLabCut](https://github.com/DeepLabCut/DeepLabCut)
- [DeepLabCut's Documentation: User Guide for Single Animal projects](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html)


This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

### This notebook assumes you already have a project folder with labeled data!

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


In [1]:
!pip install git+https://github.com/DeepLabCut/DeepLabCut.git


  Cloning https://github.com/DeepLabCut/DeepLabCut.git to /tmp/pip-req-build-lvomb2xt
  Running command git clone --filter=blob:none --quiet https://github.com/DeepLabCut/DeepLabCut.git /tmp/pip-req-build-lvomb2xt
  Resolved https://github.com/DeepLabCut/DeepLabCut.git to commit 5fb2525df9f8b49cf1d270c486fac5c2c66e4357
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 535.7 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**(Be sure to click "RESTART RUNTIME" if it is displayed above before moving on !)** You will see this button at the output of the cells above ^.

In [1]:
# Import necessary libraries
import torch
import deeplabcut

Loading DLC 3.0.0rc12...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


## Link your Google Drive (with your labeled data, or the demo data):

### First, place your project folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [21]:
# Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: `/content/drive/My Drive/yourProjectFolderName`
or were you choose to place the project folder

Tip: Using the folder icon on the left hand panel, you can navigate to where your DLC project folder is located, and copy-paste the path to the config.yaml file directly.


In [3]:
# Define the path to the config file
config = "/content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/config.yaml"

## Create a training dataset:

### You must do this step inside of Colab

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models-pytorch** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pytorch_config.yaml**.

Now it is the time to start training the network!

Tip: Setting the shuffle when creating a test-train dataset split lets us use the same test-train split repeatedly if needed, e.g. if benchmarking different model architectures on the same dataset, allowing a fair comparison of model performance.


In [7]:
shuffle = 1001

In [ ]:
# There are many more functions you can set here, including which network to use!
# Check the docstring for `create_training_dataset` for all options you can use!


deeplabcut.create_training_dataset(
    config,
    net_type="resnet_50",
    engine=deeplabcut.Engine.PYTORCH,
    Shuffles=[shuffle]  #specify which shuffle index you want - if left empty it will use the next available shuffle
)

ValueError: Cannot create shuffle 1001 as it already exists - you must either create the dataset with `userfeedback=False` or delete the shuffle with index 1001 manually (in `dlc-models`/`dlc-models-pytorch` and in the `training-datasets` folder) if you want to create a new shuffle with that index. You can otherwise create a shuffle with a new index. Existing indices are [1001].

In [ ]:
# Train the model using the specified config file

# Let's also change the display and save_epochs just in case Colab takes away
# the GPU... If that happens, you can reload from a saved point using the
# `snapshot_path` argument to `deeplabcut.train_network`:
#   deeplabcut.train_network(..., snapshot_path="/content/.../snapshot-050.pt")

# Typically, you want to train to ~200 epochs. We set the batch size to 8 to
# utilize the GPU's capabilities.

deeplabcut.train_network(
    config,
    shuffle=shuffle,
    save_epochs=5, #saves an instance of your model every 5 epochs
    epochs=200, #tells you how many epochs (rounds of viewing the whole train dataset) the training session will run for
    batch_size=8,
)


# This will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end, or when the COLAB session ends, whichever comes first.

Training with configuration:
data:
  bbox_margin: 20
  colormode: RGB
  inference:
    normalize_images: True
  train:
    affine:
      p: 0.5
      rotation: 30
      scaling: [0.5, 1.25]
      translation: 0
    crop_sampling:
      width: 448
      height: 448
      max_shift: 0.1
      method: hybrid
    gaussian_noise: 12.75
    motion_blur: True
    normalize_images: True
device: auto
metadata:
  project_path: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run
  pose_config_path: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/dlc-models-pytorch/iteration-0/openfieldOct30-trainset95shuffle1001/train/pytorch_config.yaml
  bodyparts: ['snout', 'leftear', 'rightear', 'tailbase']
  unique_bodyparts: []
  individuals: ['animal']
  with_identity: None
method: bu
model:
  backbone:
    type: ResNet
    model_name: resnet50_gn
    output_stride: 16
    freeze_bn_stats: False
    freeze_bn_weights: False
  backbone_output_channel

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

[timm/resnet50_gn.a1h_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
Data Transforms:
  Training:   Compose([
  Affine(always_apply=False, p=0.5, interpolation=1, mask_interpolation=0, cval=0, mode=0, scale={'x': (0.5, 1.25), 'y': (0.5, 1.25)}, translate_percent=None, translate_px={'x': (0, 0), 'y': (0, 0)}, rotate=(-30, 30), fit_output=False, shear={'x': (0.0, 0.0), 'y': (0.0, 0.0)}, cval_mask=0, keep_ratio=True, rotate_method='largest_box'),
  PadIfNeeded(always_apply=True, p=1.0, min_height=448, min_width=448, pad_height_divisor=None, pad_width_divisor=None, position=PositionType.CENTER, border_mode=0, value=None, mask_value=None),
  KeypointAwareCrop(always_apply=True, p=1.0, width=448, height=448, max_shift=0.1, crop_sampling='hybrid'),
  MotionBlur(always_apply=False, p=0.5, blur_limit=(3, 7), allow_shifted=True),
  GaussNoise(always_apply=False, p=0.5, var_limit=(0, 162.5625), per_channel=True, mean=0),
  Nor

KeyboardInterrupt: 

In [7]:
# This code is meant forcontinuing training from a previous snapshot


deeplabcut.train_network(
    config,
    shuffle=shuffle,
    save_epochs=5, #saves an instance of your model every 5 epochs
    epochs=200, #tells you how many epochs (rounds of viewing the whole train dataset) the training session will run for
    batch_size=8,
    snapshot_path="/content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/dlc-models-pytorch/iteration-0/openfieldOct30-trainset95shuffle1001/train/snapshot-150.pt" #replace with your snapshot path
)


Training with configuration:
data:
  bbox_margin: 20
  colormode: RGB
  inference:
    normalize_images: True
  train:
    affine:
      p: 0.5
      rotation: 30
      scaling: [0.5, 1.25]
      translation: 0
    crop_sampling:
      width: 448
      height: 448
      max_shift: 0.1
      method: hybrid
    gaussian_noise: 12.75
    motion_blur: True
    normalize_images: True
device: auto
metadata:
  project_path: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run
  pose_config_path: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/dlc-models-pytorch/iteration-0/openfieldOct30-trainset95shuffle1001/train/pytorch_config.yaml
  bodyparts: ['snout', 'leftear', 'rightear', 'tailbase']
  unique_bodyparts: []
  individuals: ['animal']
  with_identity: None
method: bu
model:
  backbone:
    type: ResNet
    model_name: resnet50_gn
    output_stride: 16
    freeze_bn_stats: False
    freeze_bn_weights: False
  backbone_output_channel

KeyboardInterrupt: 

Note, that **if you hit "STOP" you will get a `KeyboardInterrupt` "error"! No worries! :)**

## Start evaluating:
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results-pytorch**

In [8]:
# Evaluate the trained model
deeplabcut.evaluate_network(config, Shuffles=[shuffle], plotting=True)

Evaluation scorer: DLC_Resnet50_openfieldOct30shuffle1001_snapshot_best-160


100%|██████████| 6/6 [00:00<00:00, 43.23it/s]


Evaluation results file: DLC_Resnet50_openfieldOct30shuffle1001_snapshot_best-160-results.csv
Evaluation results for DLC_Resnet50_openfieldOct30shuffle1001_snapshot_best-160-results.csv (pcutoff: 0.4):
train rmse              1.16
train rmse_pcutoff      1.16
train mAP              99.50
train mAR              99.55
test rmse               1.20
test rmse_pcutoff       1.20
test mAP              100.00
test mAR              100.00
Name: (0.95, 1001, 160, -1, 0.4), dtype: float64


Here you want to see a low pixel error (RMSE)! Of course, it can only be as good as the labeler, so be sure your labels are good!

RMSE is the average Euclidean error (in pixels) between the predicted keypoint locations and the ground truth labels on a test set.
Lower RMSE = better accuracy.
For example, an RMSE of 2 px means, on average, predictions are off by 2 pixels from the true label.

mAP uses precision and recall
- Precision measures how many of the predicted keypoints are correct (i.e., how accurate your predictions are). (within a given cut-off)
- Recall measures how many of the actual keypoints were successfully detected.

Average precision (AP) is essentially the area under the precision-recall curve for that specific keypoint.
After computing AP for each keypoint, you take the mean across all keypoints
mAP ranges from 0 → 1 (or sometimes reported as 0–100%).
Higher mAP = better.
Example: mAP = 0.95 means the network finds the correct location with high confidence most of the time.

In [5]:
video1 = "/content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4"


In [18]:
# Apply trained model to videos

deeplabcut.analyze_videos(
    config,
    [video1],
    save_as_csv=True,
    shuffle=shuffle
)

Analyzing videos with /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/dlc-models-pytorch/iteration-0/openfieldOct30-trainset95shuffle1001/train/snapshot-best-160.pt
Using scorer: DLC_Resnet50_openfieldOct30shuffle1001_snapshot_best-160
Starting to analyze /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4
Video metadata: 
  Overall # of frames:    2330
  Duration of video [s]:  77.67
  fps:                    30.0
  resolution:             w=640, h=480

Running pose prediction with batch size 4


100%|██████████| 2330/2330 [01:06<00:00, 35.22it/s]


Saving results in /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4DLC_Resnet50_openfieldOct30shuffle1001_snapshot_best-160.h5 and /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4DLC_Resnet50_openfieldOct30shuffle1001_snapshot_best-160_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.



'DLC_Resnet50_openfieldOct30shuffle1001_snapshot_best-160'

In [19]:
# Create labeled videos
deeplabcut.create_labeled_video(config, [video1], shuffle=shuffle)

Starting to process video: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4
Loading /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4 and data.
Duration of video [s]: 77.67, recorded with 30.0 fps!
Overall # of frames: 2330 with cropped frame dimensions: 640 480
Generating frames and creating video.


100%|██████████| 2330/2330 [00:16<00:00, 143.87it/s]


[True]

In [12]:
#The simplest:
deeplabcut.create_labeled_video(config, [video1], save_frames=False, shuffle=shuffle)


Starting to process video: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4
Loading /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4 and data.
Duration of video [s]: 77.67, recorded with 30.0 fps!
Overall # of frames: 2330 with cropped frame dimensions: 640 480
Generating frames and creating video.


100%|██████████| 2330/2330 [00:23<00:00, 97.60it/s]


[True]

In [17]:
# Extra options for creating labelled video

#Dragged points (trailpoints):
deeplabcut.create_labeled_video(config, [video1], videotype=".mp4", trailpoints=10, shuffle=shuffle)

#Draw skeleton (requires having defined a skeleton in the config.yaml file, i.e. how different key points are connected):
deeplabcut.create_labeled_video(config, [video1], videotype=".mp4", draw_skeleton=True, shuffle=shuffle)


Starting to process video: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4
Loading /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4 and data.
Labeled video already created. Skipping...
Starting to process video: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4
Loading /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4 and data.
Labeled video already created. Skipping...
Starting to process video: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4
Loading /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4 and data.
Labeled video already created. Skipping...
Starting to process video: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run/videos/m3v1mp4.mp4
Loading /content/drive/

[None]

If model needs further refinement, I suggest after creating the videos, to download the entire model folder - open it in the GUI. In principle you should be able to used the analyzed video, but sometimes it can miss. In that case, use the original video, analyze the video anew, and then run extract frames in the GUI. Correct the labels as necessary. Upload the resulting project to Google Colab.
Set the config to your new project

In [24]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [25]:
shuffle = 1005
# Define the path to the config file
config = "/content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run-outliersExtracted_TrainingDataCreated/config.yaml"

deeplabcut.create_training_dataset(
    config,
    net_type="resnet_50",
    engine=deeplabcut.Engine.PYTORCH,
    Shuffles=[shuffle]  #specify which shuffle index you want - if left empty it will use the next available shuffle
)

[(0.95,
  1005,
  (array([  6, 107, 123,  72,  47,  49, 116,  57,   9,  94, 112,  26,  13,
           41, 118,  17,  58,  63, 101, 125,  90, 131,  64,  85, 108, 111,
          124, 105,  82, 104,  83,  54,  14,  95,  96, 121,  31, 114,  11,
          103,   2,  92,  21,  75,  53,  59,  62, 129,   0,  86,  93,   8,
          119,  61, 115,  70,  89,  36,  79,  34,  98,  43,  91,  78,  50,
           25,  67,   1,  73,  69,  76,  33,  68,  65,  28,   5,  48,  97,
           87,  45, 110,  55, 117, 100, 126,  84,  46,  80,  52, 127,  39,
           32,  24,  60,  71, 134, 122,  77, 128,  10, 135, 102,  88,   7,
           30, 132, 113,  22,  51,   4,  27,  40, 106,  37,  74, 133,  38,
           44,  81,   3,  20,  66,  56, 109,  99,  12,  23,  16,  19]),
   array([120,  35,  29, 130,  42,  18,  15])))]

In [ ]:
deeplabcut.train_network(
    config,
    shuffle=shuffle,
    save_epochs=5, #saves an instance of your model every 5 epochs
    epochs=200, #tells you how many epochs (rounds of viewing the whole train dataset) the training session will run for
    batch_size=8,
)

Training with configuration:
data:
  bbox_margin: 20
  colormode: RGB
  inference:
    normalize_images: True
  train:
    affine:
      p: 0.5
      rotation: 30
      scaling: [0.5, 1.25]
      translation: 0
    crop_sampling:
      width: 448
      height: 448
      max_shift: 0.1
      method: hybrid
    gaussian_noise: 12.75
    motion_blur: True
    normalize_images: True
device: auto
metadata:
  project_path: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run-outliersExtracted_TrainingDataCreated
  pose_config_path: /content/drive/MyDrive/Almería ML workshop/openfield-Pranav-2018-10-30_not_run-outliersExtracted_TrainingDataCreated/dlc-models-pytorch/iteration-1/openfieldOct30-trainset95shuffle1005/train/pytorch_config.yaml
  bodyparts: ['snout', 'leftear', 'rightear', 'tailbase']
  unique_bodyparts: []
  individuals: ['animal']
  with_identity: None
method: bu
model:
  backbone:
    type: ResNet
    model_name: resnet50_gn
    output_stride: 16
    f

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

[timm/resnet50_gn.a1h_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
Data Transforms:
  Training:   Compose([
  Affine(always_apply=False, p=0.5, interpolation=1, mask_interpolation=0, cval=0, mode=0, scale={'x': (0.5, 1.25), 'y': (0.5, 1.25)}, translate_percent=None, translate_px={'x': (0, 0), 'y': (0, 0)}, rotate=(-30, 30), fit_output=False, shear={'x': (0.0, 0.0), 'y': (0.0, 0.0)}, cval_mask=0, keep_ratio=True, rotate_method='largest_box'),
  PadIfNeeded(always_apply=True, p=1.0, min_height=448, min_width=448, pad_height_divisor=None, pad_width_divisor=None, position=PositionType.CENTER, border_mode=0, value=None, mask_value=None),
  KeypointAwareCrop(always_apply=True, p=1.0, width=448, height=448, max_shift=0.1, crop_sampling='hybrid'),
  MotionBlur(always_apply=False, p=0.5, blur_limit=(3, 7), allow_shifted=True),
  GaussNoise(always_apply=False, p=0.5, var_limit=(0, 162.5625), per_channel=True, mean=0),
  Nor

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
